# Optical Interference Topography
Przewidywanie odległości względem obiektu na podstawie obrazów.
Autorzy: 
- Julia Chałasiak 136531
- Grzegorz Otworowski 136605


# Ładowanie modułów
---



In [ ]:
!pip install keras-resnet

## Importy

In [ ]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import tensorflow as tf
import keras
import matplotlib.pyplot as plt 
import numpy as np
import os
import PIL
import PIL.Image
import cv2
import pandas as pd
import sys

# Pobieranie danych 
Załadowanie danych z dysku Google. Czas: ~5 min.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls gdrive/MyDrive/ZED
!unrar x gdrive/MyDrive/ZED/data.rar

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19002.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19003.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19004.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19005.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19006.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19007.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19008.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19009.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19010.png             79%  OK 
Extracting  05deg_per_s_6fps_24000frames#1/1channel/19011.png             79%  OK 
Extracting  05d

In [ ]:
!mv '05deg_per_s_6fps_24000frames#1/1channel' 'dataimages'

In [ ]:
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = "jchalasiak"
os.environ['KAGGLE_KEY'] = "364322c1591052db4c9743a12f0f764c"
!kaggle competitions download -c optical-interference-topography

  0% 0.00/152k [00:00<?, ?B/s]
100% 152k/152k [00:00<00:00, 73.7MB/s]
  0% 0.00/82.0k [00:00<?, ?B/s]
100% 82.0k/82.0k [00:00<00:00, 19.1MB/s]
  0% 0.00/140k [00:00<?, ?B/s]
100% 140k/140k [00:00<00:00, 45.9MB/s]


# Wczytywanie danych treningowych
Korzystamy z generatora (duża oszczędność wykorzystania RAM), który zmienia rozmiar oryginalnych obrazów na 224x224 oraz normalizuje je.



In [ ]:
def fix_filename(id):
  filename = str(id)
  length = len(filename)
  if length == 1:
    fixed_filename = '0000'+ filename + '.png'
  elif length == 2:
    fixed_filename = '000'+ filename + '.png'
  elif length == 3:
    fixed_filename = '00'+ filename + '.png'
  elif length == 4:
    fixed_filename = '0'+ filename + '.png'
  elif length == 5:
    fixed_filename = filename + '.png'
  return fixed_filename

In [ ]:
train_df = pd.read_csv("train.csv")
validation_df = pd.read_csv('test.csv', sep='/', decimal=',').astype(int)

fixed_filenames = train_df['id'].apply(fix_filename)
train_df['id'] = fixed_filenames

fixed_filenames = validation_df['id'].apply(fix_filename)
validation_df['id'] = fixed_filenames

In [ ]:
data_dir = '/content/dataimages'
datagen = ImageDataGenerator(rescale=1./255,
                             dtype='float16')
train_gen = datagen.flow_from_dataframe(dataframe=train_df,
                                        directory=data_dir,
                                        x_col='id',
                                        y_col='epsilon',
                                        has_ext=True,
                                        target_size=(224,224),
                                        color_mode='rgb',
                                        class_mode='other',
                                        batch_size=32)
validation_gen = datagen.flow_from_dataframe(dataframe=validation_df,
                                             directory=data_dir,
                                             x_col='id',
                                             has_ext=True,
                                             shuffle=False,
                                             target_size=(224,224),
                                             color_mode='rgb',
                                             class_mode=None,
                                             batch_size=32)

Found 12000 validated image filenames.
Found 12000 validated image filenames.


# ResNet
W projekcie wykorzystaliśmy jako bazę ResNet50 bez ostatniej warstwy oraz zmieniliśmy funkcję straty na MSE. 

## Tworzenie modelu
W budowie modelu wykorzystaliśmy ResNet jako bazę, a także warstwy z funkcją aktywacji ReLu, 'dropout layers' mające uchronić przed przeuczeniem oraz 'batch layers' mające za zadanie normalizację danych wejściowych.



In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))

base_model = ResNet50(include_top=False,
                 input_tensor=inputs,
                 weights=None)


base_model.trainable = False

batch_norm_l1 = tf.keras.layers.BatchNormalization()
batch_norm_l2 = tf.keras.layers.BatchNormalization()
batch_norm_l3 = tf.keras.layers.BatchNormalization()
batch_norm_l4 = tf.keras.layers.BatchNormalization()

flatten_l = tf.keras.layers.Flatten()

dense_l1 = tf.keras.layers.Dense(256, activation='relu')
dense_l2 = tf.keras.layers.Dense(128, activation='relu')
dense_l3 = tf.keras.layers.Dense(64, activation='relu')

dropout_l1 = tf.keras.layers.Dropout(0.5)
dropout_l2 = tf.keras.layers.Dropout(0.5)
dropout_l3 = tf.keras.layers.Dropout(0.5)

linear_l = tf.keras.layers.Dense(1, activation="linear")


x = base_model(inputs)
x = flatten_l(x)
x = batch_norm_l1(x)

x = dense_l1(x)
x = dropout_l1(x)
x = batch_norm_l2(x)

x = dense_l2(x)
x = dropout_l2(x)
x = batch_norm_l3(x)

x = dense_l3(x)
x = dropout_l3(x)
x = batch_norm_l4(x)

outputs = linear_l(x)


model = tf.keras.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='mse')

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 batch_normalization_4 (Batc  (None, 100352)           401408    
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 256)               25690368  
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                           

## Trenowanie modelu
W trenowaniu wykorzystaliśmy koncepcję zamrażania i odmrażania warstw dodanych do ResNeta w celu przyspieszenia uczenia. Po każdej takiej zmianie model wymagał kompilacji. 

In [ ]:
N_epochs = 6

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

history_1 = model.fit(train_gen, 
                      epochs=N_epochs,
                      callbacks=[callback])

dense_l1.trainable = False
dense_l2.trainable = False
dense_l3.trainable = False
base_model.trainable = True

model.compile(tf.keras.optimizers.Adam(),
              loss='mse')

history_2 = model.fit(train_gen, 
                      epochs=N_epochs,
                      callbacks=[callback])

dense_l1.trainable = True
dense_l2.trainable = True
dense_l3.trainable = True

model.compile(tf.keras.optimizers.Adam(),
              loss='mse')

history_3 = model.fit(train_gen, 
                      epochs=N_epochs,
                      callbacks=[callback])

Epoch 1/6
375/375 [==============================] - 107s 273ms/step - loss: 0.6148
Epoch 2/6
375/375 [==============================] - 101s 269ms/step - loss: 0.0790
Epoch 3/6
375/375 [==============================] - 103s 273ms/step - loss: 0.0363
Epoch 4/6
375/375 [==============================] - 101s 269ms/step - loss: 0.0279
Epoch 5/6
375/375 [==============================] - 101s 269ms/step - loss: 0.0242
Epoch 6/6
375/375 [==============================] - 101s 268ms/step - loss: 0.0214
Epoch 1/6
375/375 [==============================] - 272s 704ms/step - loss: 0.0210
Epoch 2/6
375/375 [==============================] - 264s 703ms/step - loss: 0.0166
Epoch 3/6
375/375 [==============================] - 264s 703ms/step - loss: 0.0164
Epoch 4/6
375/375 [==============================] - 264s 703ms/step - loss: 0.0152
Epoch 5/6
375/375 [==============================] - 264s 703ms/step - loss: 0.0148
Epoch 6/6
375/375 [==============================] - 264s 702ms/step - loss:

## Zapisywanie modelu

In [ ]:
model.save('model_06')
!zip -r /content/model_06.zip /content/model_06

INFO:tensorflow:Assets written to: model_06/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


  adding: content/model_06/ (stored 0%)
  adding: content/model_06/variables/ (stored 0%)
  adding: content/model_06/variables/variables.index (deflated 80%)
  adding: content/model_06/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/model_06/keras_metadata.pb (deflated 96%)
  adding: content/model_06/saved_model.pb (deflated 92%)
  adding: content/model_06/assets/ (stored 0%)


## Wczytanie modelu
W sytuacji gdy chcemy wczytać wcześniej zapisany model.

In [ ]:
# model = keras.models.load_model('model_04')

# Testowanie modelu

## Predykcja

In [ ]:
y_test = model.predict(validation_gen)

In [ ]:
y_test = y_test.clip(min=0.0, max=1.0)

## Wyświetlenie (dataframe)

In [ ]:
id = range(12000, 24000)
df = pd.DataFrame(y_test, index=id)
df = df.set_axis(['epsilon'], axis = 1)
df

,epsilon
12000,0.354506
12001,0.258264
12002,0.165114
12003,0.092164
12004,0.000000
...,...
23995,0.821742
23996,0.848701
23997,0.796036
23998,0.713785


## Zapis do csv

In [ ]:
df.to_csv("results.csv", index_label='id')